In [1]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Settings,
    ServiceContext,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.embeddings.utils import resolve_embed_model
from llama_index.llms.ollama import Ollama
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

In [2]:
import json

In [3]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [4]:
# %conda install -c conda-forge ipywidgets

In [5]:
# documents = SimpleDirectoryReader("data/docs").load_data()

# embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")

# service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)

# index = VectorStoreIndex.from_documents(
#     documents, show_progress=True, service_context=service_context
# )

# index.storage_context.persist(persist_dir="data/persist_dir")

In [4]:
from llama_index.core import Settings

In [6]:
llm = Ollama(model="mistral", base_url="http://62.80.172.138:11434", request_timeout=180.0)

In [9]:
response = llm.complete("What is the capital of France?")
print(response)

KeyboardInterrupt: 

In [7]:

Settings.embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")

In [8]:
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="data/persist_dir")

# load index
index = load_index_from_storage(storage_context, show_progress=True) # , service_context=service_context

In [9]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=1,
)

In [10]:
def escape_curly_braces(input_string: str) -> str:
    # Replace '{' with '{{' and '}' with '}}' to escape curly braces
    escaped_string = input_string.replace("{", "{{").replace("}", "}}")
    return escaped_string

In [11]:
# user_prompt_long = """Sentence: Set the kitchen temperature to 25 degrees, adjust the humidity in the living room to 50%,
# turn on the lights in the bedroom, and turn off the fan in the bathroom.
# JSON: """

# user_prompt_very_long = """Sentence: Turn on the lights in the kitchen, set the living room temperature to 22 degrees,
# close the curtains in the bedroom, set the bathroom fan to auto mode, turn off the desk lamp in the office,
# close the garage door, water the garden manually, dim the lights on the bookshelves in the library,
# turn on the air conditioning in the gym, and put the computer in the study to sleep.
# JSON: """

json_scheme_general = {
    "method": {
        "type": "string"
    },
    "params": {
        "type": "object"
    }
}

example_1_json = {
  "method":"Cover.Open",
  "params":
  {
    "id":2
  }
}

example_2_json = {
  "method":"Cover.GetStatus",
  "params":
  {
    "id":0
  }
}

variables = {
    "instruction": """You are 'Al', a helpful AI Assistant that controls the devices in a house. Create a JSON object representing given command. Strictly output in JSON format.""",
    "json_scheme": "The output JSON should follow the next scheme: " + json.dumps(json_scheme_general),
    "devices": """Cover id=1""",
    "example_1": """Devices: Cover id=2
Functions: Name: Cover.Open
Description: Preconditions: Cover will not accept the command if: An overvoltage error is set at the time of the request. An undervoltage error is set at the time of the request. An overtemp error is set at the time of the request. An engaged safety_switch prohibits movement in the requested direction. Cover calibration is running at the time of the request. Properties: [{'name': 'id', 'type': 'number', 'description': 'The numeric ID of the Cover component instance'}, {'name': 'duration', 'type': 'number', 'description': 'If duration is not provided, Cover will fully open, unless it times out because of maxtime_open first. If duration (seconds) is provided, Cover will move in the open direction for the specified time. duration must be in the range [0.1..maxtime_open]Optional'}] Response: null on success; error if the request can not be executed or failed 
Command: Open the cover.
JSON: """ + json.dumps(example_1_json),
    "example_2": """Devices: Cover id=0
Functions: Name: Cover.GetStatus
Description: Properties: [{'name': 'id', 'type': 'number', 'description': 'The numeric ID of the Cover component instance'}] Find more about the status properties in status section 
Command: Get cover status.
JSON: """ + json.dumps(example_2_json),
}

In [12]:
prompt_template = """
{instruction}
{json_scheme}

{example_1}

{example_2}
"""

final_prompt = prompt_template.format(**variables) # escape_curly_braces()

# final_prompt = variables['user_prompt'] + '{cont}'

print(final_prompt)


You are 'Al', a helpful AI Assistant that controls the devices in a house. Create a JSON object representing given command. Strictly output in JSON format.
The output JSON should follow the next scheme: {"method": {"type": "string"}, "params": {"type": "object"}}

Devices: Cover id=2
Functions: Name: Cover.Open
Description: Preconditions: Cover will not accept the command if: An overvoltage error is set at the time of the request. An undervoltage error is set at the time of the request. An overtemp error is set at the time of the request. An engaged safety_switch prohibits movement in the requested direction. Cover calibration is running at the time of the request. Properties: [{'name': 'id', 'type': 'number', 'description': 'The numeric ID of the Cover component instance'}, {'name': 'duration', 'type': 'number', 'description': 'If duration is not provided, Cover will fully open, unless it times out because of maxtime_open first. If duration (seconds) is provided, Cover will move in t

In [13]:
# from llama_index.core import PromptTemplate

# qa_prompt = PromptTemplate(final_prompt)

In [14]:
# configure response synthesizer
# response_synthesizer = get_response_synthesizer(text_qa_template=qa_prompt) # service_context=service_context

In [15]:
# from llama_index.core.response_synthesizers.base import BaseSynthesizer

In [16]:
# from typing import Sequence, Any

# class EnvironmentSynthesizer(BaseSynthesizer):
#     def get_response(
#         self,
#         query_str: str,
#         text_chunks: Sequence[str],
#         **response_kwargs: Any,
#     ) -> RESPONSE_TEXT_TYPE:
#         """Get response."""
#         ...

In [17]:
# assemble query engine
# query_engine = RetrieverQueryEngine(
#     retriever=retriever,
#     response_synthesizer=response_synthesizer,
    # node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
# )

In [18]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger()
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

In [19]:
# logger.disabled = True

In [20]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext,
)
from IPython.display import Markdown, display

In [18]:
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [19]:
prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>


You are 'Al', a helpful AI Assistant that controls the devices in a house. Create a JSON object representing given command. Strictly output in JSON format.
The output JSON should follow the next scheme: {{"method": {{"type": "string"}}, "params": {{"type": "object"}}}}

Devices: Cover id=2
Functions: Name: Cover.Open
Description: Preconditions: Cover will not accept the command if: An overvoltage error is set at the time of the request. An undervoltage error is set at the time of the request. An overtemp error is set at the time of the request. An engaged safety_switch prohibits movement in the requested direction. Cover calibration is running at the time of the request. Properties: [{{'name': 'id', 'type': 'number', 'description': 'The numeric ID of the Cover component instance'}}, {{'name': 'duration', 'type': 'number', 'description': 'If duration is not provided, Cover will fully open, unless it times out because of maxtime_open first. If duration (seconds) is provided, Cover will 

<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [21]:
import pandas as pd
import random

In [22]:
user_prompt_template = """Devices: {env}
Methods:
{methods_description}
Command: {user_cmd}
JSON:
"""

In [27]:
limit_rows = 10
df = pd.read_csv('dataset_v0.csv')
devices = list(df['device'].unique())
output_df = pd.DataFrame(columns=['device', 'user_cmd', 'mtd', 'json_cmd'])

for i, row in df.iterrows():
    user_cmd = row['user_cmd']

    device = row['device']
    sample_devices = devices.copy()
    sample_devices.remove(device)
    sample_devices = random.sample(sample_devices, k=2)
    env = f'{sample_devices[0]} id=1, {sample_devices[1]} id=2, {device} id=444'

    retrieved_nodes = retriever.retrieve(user_cmd)
    methods_description = f'API method 1:\n{retrieved_nodes[0].text}'
    method = retrieved_nodes[0].metadata['file_name'].replace('.md', '')

    user_prompt = user_prompt_template.format(**{'env': env, 
                                                 'methods_description': methods_description, 
                                                 'user_cmd': user_cmd})
    query = final_prompt + '\n\n' + user_prompt
    response_text = llm.complete(query).text
    print(response_text)
    json_cmd = json.dumps(json.loads(response_text))

    output_df.loc[len(output_df)] = pd.Series({'device': row['device'], 'user_cmd': user_cmd, 'mtd': method, 'json_cmd': json_cmd})

    if i == limit_rows - 1:
        break
output_df.to_csv('output.csv', index=False)

DEBUG:llama_index.core.indices.utils:> Top 1 nodes:
> [Node f059ed09-ff31-4d85-ba1c-67e2bf64d1f1] [Similarity score:             0.684745] Method name: Switch.SetConfig
Method description: Properties:
{"id": {"type": "number", "descript...
> Top 1 nodes:
> [Node f059ed09-ff31-4d85-ba1c-67e2bf64d1f1] [Similarity score:             0.684745] Method name: Switch.SetConfig
Method description: Properties:
{"id": {"type": "number", "descript...
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/vpo/miniconda3/envs/assistant/lib/python3.10/site-packages/certifi/cacert.pem'
load_verify_locations cafile='/home/vpo/miniconda3/envs/assistant/lib/python3.10/site-packages/certifi/cacert.pem'
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=11434 local_address=None timeout=180.0 socket_options=None
connect_tcp.started host='localhost' port=11

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Sun, 03 Mar 2024 18:28:13 GMT'), (b'Transfer-Encoding', b'chunked')])
receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Sun, 03 Mar 2024 18:28:13 GMT'), (b'Transfer-Encoding', b'chunked')])
INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
response_closed.started
DEBUG:httpcore.http11:response_closed.complete
response_closed.complete
DEBUG:httpcore.connection:close.sta

JSONDecodeError: Invalid \escape: line 1 column 74 (char 73)

In [26]:
import ast

output_df = pd.read_csv('output.csv')
incorrect_output_df = pd.DataFrame(columns=['device', 'user_cmd', 'gt_mtd', 'output_mtd', 'gt_json_cmd', 'output_json_cmd'])

correct_methods = 0
correct_json_cmds = 0
for i in range(len(output_df)):
    gt = df.iloc[i]
    output = output_df.iloc[i]

    correct_ouput = True
    if gt['mtd'] == output['mtd']:
        correct_methods += 1
    else:
        correct_ouput = False
    try:
        if ast.literal_eval(gt['json_cmd'].replace("'", '"')) == json.loads(output['json_cmd']):
            correct_json_cmds += 1
        else:
            correct_ouput = False
    except Exception as ex:
        print(ex)
        print(gt['json_cmd'].replace("'", '"'))
        print(i, output['json_cmd'])
    
    if not correct_ouput:
        incorrect_output_df.loc[len(incorrect_output_df)] = pd.Series({'device': gt['device'], 'user_cmd': gt['user_cmd'], 'gt_mtd': gt['mtd'],
                                                              'output_mtd': output['mtd'], 'gt_json_cmd': gt['json_cmd'], 'output_json_cmd': output['json_cmd']})
acc_methods = round(correct_methods / len(output_df), 2)
acc_json_cmds = round(correct_json_cmds / len(output_df), 2)

incorrect_output_df.to_csv('incorrect_output.csv', index=False)

with open('results', 'a') as f:
    f.write(f'Acc of methods: {acc_methods}\n'
            f'Acc of json cmds: {acc_json_cmds}\n\n'
            f'-----------------------------------\n\n')

In [36]:
response.response

' {\n"method": "Cover.Open",\n"params": {\n"id": 2\n}\n}'

In [5]:
a = {0: {1: 2}}
b = {0: {1: 2}}
a == b

True